# Imports

In [1]:
# coding: utf-8

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')



C:\Users\kmura\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import cv2


# Object detection imports
Here are the imports from the object detection module.


In [3]:

from utils import label_map_util

from utils import visualization_utils as vis_util



D:\IIIT D\Computer Vision\Obj Detection\models-master\research\object_detection\utils\visualization_utils.py:27: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')



## Download Model

In [25]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.


In [5]:

MODEL_NAME='ssd_mobilenet_v1_coco_2017_11_17'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
	od_graph_def = tf.GraphDef()
	with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
		serialized_graph = fid.read()
		od_graph_def.ParseFromString(serialized_graph)
		tf.import_graph_def(od_graph_def, name='')



## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.	 Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine


In [51]:


category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)



In [52]:
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

## Helper code


In [8]:


def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
	  (im_height, im_width, 3)).astype(np.uint8)



## Detection

In [9]:
def run_inference_for_single_image(image, graph):
	with graph.as_default():
		with tf.Session() as sess:
			# Get handles to input and output tensors
			ops = tf.get_default_graph().get_operations()
			all_tensor_names = {output.name for op in ops for output in op.outputs}
			tensor_dict = {}
			for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
				tensor_name = key + ':0'
				if tensor_name in all_tensor_names:
					tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
	
			if 'detection_masks' in tensor_dict:
			# The following processing is only for single image
				detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
				detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
			# Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
				real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
				detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
				detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
				detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(detection_masks, detection_boxes, image.shape[0], image.shape[1])
				detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
		# Follow the convention by adding back the batch dimension
				tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
			image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

	  # Run inference
			output_dict = sess.run(tensor_dict,feed_dict={image_tensor: np.expand_dims(image, 0)})

	  # all outputs are float32 numpy arrays, so convert types as appropriate
			output_dict['num_detections'] = int(output_dict['num_detections'][0])
			output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
			output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
			output_dict['detection_scores'] = output_dict['detection_scores'][0]
			if 'detection_masks' in output_dict:
				output_dict['detection_masks'] = output_dict['detection_masks'][0]
	return output_dict




In [10]:
def find_indxs(thresh, output_dict):

	for indx,elem in enumerate(output_dict['detection_scores']):

		if elem < thresh:
			break
	
	return indx


def give_classes(category_index,output_dict):
	
	break_indx = find_indxs(0.50, output_dict)

	classes= output_dict['detection_classes'][:break_indx]
	classes= classes.tolist()
	
	scores = output_dict['detection_scores'][:break_indx]
	scores=scores.tolist()

	windows = output_dict['detection_boxes'][: break_indx]
	windows=windows.tolist()
	
	objects=[]
	
	for score,clas in zip(scores,classes):
		
		objects.append([category_index[clas]['name'],score])
		
	return objects
		




In [11]:
import pickle

# Video_Footage folder is the main one

 "videos" folder contains only videos

 "video_obj_detects" folder contains folders of videos and each video folder has "pickle" files containing variable, which is a list of objects in that frame.

 "video_frames" folder contains folders of videos and each video folder has multiple frames.



In [12]:
import os


video_dataset_path = './VIDEO_FOOTAGE/'

video_names_formats = os.listdir("{}/videos/".format(video_dataset_path))

In [13]:
video_names_formats

['test.mp4', 'test1.mp4', 'test2.mp4']

In [14]:
video_titles = [vid_name.split(".")[0] for vid_name in video_names_formats]

# Store the list of identifiable things by model
For ex : 
1. Car
2. Human
3. Bike etc

In [36]:
object_num_mapper=dict()
for i in category_index:
    object_num_mapper[i]=category_index[i]['name']


object_mapper= dict((v,k) for (k,v) in object_num_mapper.items())

import pickle

with open("object_mapper.pickle","wb") as fh:
    pickle.dump(object_mapper,fh)

    
with open("object_num_mapper.pickle","wb") as fh:
    pickle.dump(object_num_mapper,fh)

In [24]:
from tqdm import tqdm_notebook

for vid_nam_frmt in tqdm_notebook(video_names_formats,desc='video-progress'):
    
    vidcap= cv2.VideoCapture("{}/videos/{}".format(video_dataset_path,vid_nam_frmt))
    TEST_IMAGE_PATHS=[]

    success,vid_image = vidcap.read()
    count = 0

    new_folder_path="{}/video_frames/{}".format(video_dataset_path,vid_nam_frmt)
    os.mkdir(new_folder_path)
    
    os.mkdir("{}/video_obj_detects/{}".format(video_dataset_path,vid_nam_frmt))

    while success:
        new_frame_path="{}/frame{}.jpg".format(new_folder_path,count)
        
        cv2.imwrite(new_frame_path, vid_image)      	  # save frame as JPEG file
        TEST_IMAGE_PATHS.append(new_frame_path)
        success,vid_image = vidcap.read()
        count += 1
    
    count=count-1
        
#     imgs_objects=[]


    count=0

    for image_path in tqdm_notebook(TEST_IMAGE_PATHS[:5],desc='progress',leave=False):

        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the

        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)



        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)

        # Actual detection.
        output_dict = run_inference_for_single_image(image_np, detection_graph)
        objects = give_classes(category_index,output_dict)
        
        
        
        new_frame_features="{}/video_obj_detects/{}/frame{}.pickle".format(video_dataset_path,vid_nam_frmt,count)
        
        with open(new_frame_features,'wb') as fh:
            pickle.dump(objects,fh)
        
        count+=1
        break
    break    
#         imgs_objects.append(objects)

    
    

# Convert to activation vectors

In [73]:
def conv2vec(object_mapper,given_vector):
    bow= max(list(object_mapper.values())) *[0]    
    identified_objs= [el[0] for el in given_vector]
    
    for v,k in object_mapper.items():
        if v in identified_objs:
            bow[k-1]=1
    
    return bow

In [74]:
conv2vec(object_mapper,objects)

['elephant']
person
bicycle
car
motorcycle
airplane
bus
train
truck
boat
traffic light
fire hydrant
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
couch
potted plant
bed
dining table
toilet
tv
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]